<a href="https://colab.research.google.com/github/yimingm/MSSP608-Practical-Machine-Learning/blob/master/Homework6_Unsupervised_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 6 (Due March 22, 2020)
In this homework we are working with a dataset of American universities, aggregated from government statistics. The dataset contains a large number of columns covering financial details, enrollment details, and so on. We will be using unsupervised dimensionality reduction to discover which features are highly related, and then using unsupervised clustering to discover which colleges are most similar.

# Initial Data Fetching and Setup

In [ ]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1lPoKPVXt9glGvBlxA-lwZDtmpyUsvIpL' -O colleges.csv


--2020-03-31 01:01:21--  https://docs.google.com/uc?export=download&id=1lPoKPVXt9glGvBlxA-lwZDtmpyUsvIpL
Resolving docs.google.com (docs.google.com)... 172.217.203.100, 172.217.203.113, 172.217.203.102, ...
Connecting to docs.google.com (docs.google.com)|172.217.203.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-b0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/pojdefo9clb7q0ucibet6kbpsid651eb/1585616475000/09640029349513786515/*/1lPoKPVXt9glGvBlxA-lwZDtmpyUsvIpL?e=download [following]
--2020-03-31 01:01:22--  https://doc-04-b0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/pojdefo9clb7q0ucibet6kbpsid651eb/1585616475000/09640029349513786515/*/1lPoKPVXt9glGvBlxA-lwZDtmpyUsvIpL?e=download
Resolving doc-04-b0-docs.googleusercontent.com (doc-04-b0-docs.googleusercontent.com)... 172.217.204.132, 2607:f8b0:400c:c15::84
Connecting to doc-04-b0-docs.googleusercontent.com (doc-04

In [ ]:
import pandas as pd
input_file = pd.read_csv("colleges.csv")
print(input_file.shape)
print(input_file.head())

input_file_clean = {
    "Institution name": input_file["Institution name"]
}
for column in input_file.loc[:, input_file.columns[1:]]:
  clean_column = []
  original_column = input_file[column].replace("-", None)
  for x in original_column:
    try:
      clean_column.append(float(x))
    except:
      clean_column.append(None)
  input_file_clean[column] = clean_column
df = pd.DataFrame(input_file_clean)
df = df.dropna()
# df.columns


(3815, 77)
                      Institution name  ... Total_Nonresident alien 
0             Alabama A & M University  ...                       70
1  University of Alabama at Birmingham  ...                     1049
2  University of Alabama in Huntsville  ...                      449
3             Alabama State University  ...                       58
4            The University of Alabama  ...                     1224

[5 rows x 77 columns]



## Question 1 *(4 points)*

The subsets and features (16 in total) I chose are as follows:

| Feature Set | Description                                                               |
|-------------|---------------------------------------------------------------------------|
| Size        | Features representing the size of a university.                |
| Selectivity | Features representing the difficulty of being admitted and/or graduating. |
| Diversity   | Features representing the breadth of  representation in the student body.              |
| Faculty   | Features representing the composition of university faculties.             |

Size: 
49.	'Undergraduate_Total',
59.	'Graduate_Total',

Selectivity:
13.	'Undergraduate total_Grand Total', 
20. 'Graduate_Grand Total',
41.	'Graduation rate (%)', 

Diversity: 
51.	'Undergraduate_Asian ', 
52.	'Undergraduate_Black or African American ',
55.	'Undergraduate_White ',
61.	'Graduate_Asian ',
62.	'Graduate_Black or African American ', 
65.	'Graduate_White ', 
21.	'All students total_Full time - Men',
22.	'All students total_Full time - Women',

Faculty:
44.	'All full-time instructional staff', 
47.	'On tenure track', 
48.	'Not on tenure track'


## Task 2. *(14 points total)*

### Task 2.1 *(2 points)*
Normalized the features in the university dataset. Each feature in the resulting data frame should be normalized to a value between 0 and 1. Save this normalized representation to a variable named `df_norm`.

In [ ]:
# Set up each feature set
size = ["Undergraduate_Total",
"Graduate_Total"]

selectivity = ["Undergraduate total_Grand Total",
"Graduate_Grand Total",
"Graduation rate (%)"]

diversity = ["Undergraduate_Asian ",
"Undergraduate_Black or African American ",
"Undergraduate_White ",
"Graduate_Asian ",
"Graduate_Black or African American ",
"Graduate_White ",
"All students total_Full time - Men",
"All students total_Full time - Women"]

faculty = ["All full-time instructional staff",
"On tenure track",
"Not on tenure track"]

In [ ]:
# Normalize features using Minmaxscaler form sklearn
from sklearn.preprocessing import MinMaxScaler
feature_set = size + selectivity + diversity + faculty
X = df.loc[:, feature_set]

print("Before Scaling:")
print(X.loc[:5])

scaler = MinMaxScaler() 
df_norm = pd.DataFrame(scaler.fit_transform(X))
df_norm.columns = X.columns
print("After Scaling")
print(df_norm.loc[:5])

Before Scaling:
   Undergraduate_Total  Graduate_Total  ...  On tenure track  Not on tenure track
1              13836.0          8087.0  ...            533.0               1056.0
2               7671.0          2065.0  ...             69.0                120.0
3               3903.0           510.0  ...             56.0                 56.0
4              33028.0          5362.0  ...            401.0                492.0
5               1681.0          5362.0  ...              8.0                  1.0

[5 rows x 16 columns]
After Scaling
   Undergraduate_Total  Graduate_Total  ...  On tenure track  Not on tenure track
0             0.167267        0.223908  ...         0.272495             0.271117
1             0.092710        0.057154  ...         0.035276             0.030809
2             0.047141        0.014095  ...         0.028630             0.014377
3             0.399369        0.148451  ...         0.205010             0.126316
4             0.020269        0.148451  ...  

### Task 2.2 Unsupervised Features *(6 points)*
Using principal component analysis on `df_norm`, generate a dimensionality-reduced representation of the university dataset. 

- Vary the number of components from 2 to 10. What percentage of variance is explained at each number of components?
![alt text](https://drive.google.com/uc?id=1wmlaiYUCElYWvKDBwKPmkr9r8FNFc5pW)
- Set the number of components to 4. Which features load together with high weight in each component?
![alt text](https://drive.google.com/uc?id=1SXLxwhjTe1xiRPICVeRou9CBGzmkr9SU)



In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
components = pca.fit_transform(df_norm)
print("Explained variance of each components:")
print(pca.explained_variance_ratio_)
print()

# cumulative components
import numpy as np
cum_var = np.cumsum(np.round(pca.explained_variance_ratio_,decimals = 4))
print("Cumulative Explained variances:")
print(cum_var)


Explained variance of each components:
[0.45437061 0.31145732 0.11397996 0.03678954 0.02665693 0.01823079
 0.0094208  0.00862613 0.00661337 0.00513798]

Cumulative Explained variances:
[0.4544 0.7659 0.8799 0.9167 0.9434 0.9616 0.971  0.9796 0.9862 0.9913]


In [ ]:
# Set the number of components to 4

factors = pd.DataFrame({
    "Features":df_norm.columns,
    "Component 1": pca.components_[0],
    "Component 2": pca.components_[1],
    "Component 3": pca.components_[2],
    "Component 4": pca.components_[3]
})

pd.set_option('display.max_columns', None)
print(factors)

                                    Features  Component 1  Component 2  \
0                        Undergraduate_Total     0.103870     0.340994   
1                             Graduate_Total     0.127064     0.269621   
2            Undergraduate total_Grand Total     0.103870     0.340994   
3                       Graduate_Grand Total     0.127064     0.269621   
4                        Graduation rate (%)     0.908885    -0.399392   
5                       Undergraduate_Asian      0.101917     0.226025   
6   Undergraduate_Black or African American      0.015474     0.202314   
7                       Undergraduate_White      0.095664     0.246524   
8                            Graduate_Asian      0.099916     0.178924   
9        Graduate_Black or African American      0.033206     0.115416   
10                           Graduate_White      0.135188     0.278327   
11        All students total_Full time - Men     0.181061     0.314295   
12      All students total_Full time -

In [ ]:
# To sum up the weight by subset
# First, add the column "subset" to the dataset "factors"

factors["subset"] = "size"
factors["subset"].iloc[2:5] = "selectivity"
factors["subset"].iloc[5:13] = "diversity"
factors["subset"].iloc[13:] = "faculty"
# print(factors)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [ ]:
# Then, sum up weight (absolute value) in each group

factors["abs_1"] = factors["Component 1"].abs()
factors["abs_2"] = factors["Component 2"].abs()
factors["abs_3"] = factors["Component 3"].abs()
factors["abs_4"] = factors["Component 4"].abs()
print("Weight of component 1 by groups:")
print(factors.groupby(["subset"])["abs_1"].sum())
print()
print("Weight of component 2 by groups:")
print(factors.groupby(["subset"])["abs_2"].sum())
print()
print("Weight of component 3 by groups:")
print(factors.groupby(["subset"])["abs_3"].sum())
print()
print("Weight of component 4 by groups:")
print(factors.groupby(["subset"])["abs_4"].sum())
print()

Weight of component 1 by groups:
subset
diversity      0.759641
faculty        0.283112
selectivity    1.139820
size           0.230935
Name: abs_1, dtype: float64

Weight of component 2 by groups:
subset
diversity      1.736317
faculty        0.402219
selectivity    1.010006
size           0.610614
Name: abs_2, dtype: float64

Weight of component 3 by groups:
subset
diversity      1.803076
faculty        0.093958
selectivity    0.761522
size           0.719102
Name: abs_3, dtype: float64

Weight of component 4 by groups:
subset
diversity      2.011017
faculty        0.867436
selectivity    0.352031
size           0.253777
Name: abs_4, dtype: float64



### Task 2.3. Unsupervised Clusters (6 points) 
Tune a K-means cluster model on this dataset, using the features from `df_norm`. In particular, tune the number of clusters *K* to a level that, in your best judgment, produces representative clusters and appropriate complexity.


In [ ]:
from sklearn.cluster import KMeans

# Print statistics about the cluster
def describe_cluster(cluster_instances):
  undergrad_size = cluster_instances["Undergraduate_Total"].mean()
  graduate_size = cluster_instances["Graduate_Total"].mean()
  undergrad_grad = cluster_instances["Undergraduate total_Grand Total"].mean()
  graduate_grad = cluster_instances["Graduate_Grand Total"].mean()
  graduation_rate = cluster_instances["Graduation rate (%)"].mean()
  undergrad_asian = cluster_instances["Undergraduate_Asian "].mean()
  undergrad_black = cluster_instances["Undergraduate_Black or African American "].mean()
  undergrad_white = cluster_instances["Undergraduate_White "].mean()
  grad_asian = cluster_instances["Graduate_Asian "].mean()
  grad_black = cluster_instances["Graduate_Black or African American "].mean()
  grad_white = cluster_instances["Graduate_White "].mean()
  men = cluster_instances["All students total_Full time - Men"].mean()
  women = cluster_instances["All students total_Full time - Women"].mean()
  staff = cluster_instances["All full-time instructional staff"].mean()
  tenure_staff = cluster_instances["On tenure track"].mean()
  nontenure_staff = cluster_instances["Not on tenure track"].mean()
  print(f"  Mean undergrad_size: {undergrad_size:.1f}")
  print(f"  Mean graduate_size: {graduate_size:.1f}")
  print(f"  Mean undergrad_grad: {undergrad_grad:.1f}")
  print(f"  Mean graduate_grad: {graduate_grad:.1f}")
  print(f"  Mean graduation_rate: {graduation_rate:.1f}")
  print(f"  Mean undergrad_asian: {undergrad_asian:.1f}")
  print(f"  Mean undergrad_black: {undergrad_black:.1f}")
  print(f"  Mean undergrad_white: {undergrad_white:.1f}")
  print(f"  Mean grad_asian: {grad_asian:.1f}")
  print(f"  Mean grad_black: {grad_black:.1f}")
  print(f"  Mean grad_white: {grad_white:.1f}")
  print(f"  Mean men: {men:.1f}")
  print(f"  Mean women: {women:.1f}")
  print(f"  Mean staff: {staff:.1f}")
  print(f"  Mean tenure_staff: {tenure_staff:.1f}")
  print(f"  Mean nontenure_staff: {nontenure_staff:.1f}")

for k in range(2,6):
    print(f"Cluster Statistics for k = {k}")

    # Learn Clusters
    learner = KMeans(n_clusters=k)
    clusters = learner.fit(df_norm)

    # Add clusters do datafram and describe the clusters
    df["cluster"] = clusters.labels_
    for cluster_id in df["cluster"].unique():
        print(f"Cluster {cluster_id}")
        cluster_instances = df.loc[df.cluster == cluster_id]
        print(f"  {100*len(cluster_instances)/len(df):.1f}% of data.")
        describe_cluster(cluster_instances)

    print("------------------------------")

Cluster Statistics for k = 2
Cluster 0
  45.6% of data.
  Mean undergrad_size: 4558.8
  Mean graduate_size: 1853.4
  Mean undergrad_grad: 4558.8
  Mean graduate_grad: 1853.4
  Mean graduation_rate: 0.7
  Mean undergrad_asian: 352.1
  Mean undergrad_black: 405.7
  Mean undergrad_white: 2476.8
  Mean grad_asian: 122.6
  Mean grad_black: 162.0
  Mean grad_white: 937.5
  Mean men: 2044.5
  Mean women: 2473.4
  Mean staff: 289.4
  Mean tenure_staff: 77.6
  Mean nontenure_staff: 115.7
Cluster 1
  54.4% of data.
  Mean undergrad_size: 4243.6
  Mean graduate_size: 1150.0
  Mean undergrad_grad: 4243.6
  Mean graduate_grad: 1150.0
  Mean graduation_rate: 0.3
  Mean undergrad_asian: 215.1
  Mean undergrad_black: 627.8
  Mean undergrad_white: 1875.9
  Mean grad_asian: 58.4
  Mean grad_black: 144.8
  Mean grad_white: 562.1
  Mean men: 850.1
  Mean women: 1089.9
  Mean staff: 95.2
  Mean tenure_staff: 45.2
  Mean nontenure_staff: 49.7
------------------------------
Cluster Statistics for k = 3
Clust

In [ ]:
# Use silhouette score to evaluate Clusters

from sklearn.metrics import silhouette_score

sils = []

for k in range(2,6):
  learner = KMeans(n_clusters=k, random_state=123)
  clusters = learner.fit(df_norm)
  df["cluster"] = clusters.labels_
  silhouette = silhouette_score(df_norm, df["cluster"])
  sils.append(silhouette)
  print(f"K={k}, silhouette={silhouette:.3f}")

K=2, silhouette=0.366
K=3, silhouette=0.394
K=4, silhouette=0.290
K=5, silhouette=0.301


When k = 3

  - cluster 0 represents universities with extremely large student cohort (both undergrad and graduate).Also the number of non-tenured staff is almost twice as the number of tenured staff.

  - cluster 1 represents universities with smaller Asian cohort.

  - cluster 2 represents universities with reletively low graduation rates.

## Question 3 *(6 points)*

   - What number of clusters did you choose, and what was your evaluation process for choosing that value?
   I chose 2 as the number of clusters, based on the value of silhouette score.
   - Do the sets of features you chose manually align with the principal components and/or clusters you discovered?
   - What additional features, not present in this data, would be useful to group together similar universities using unsupervised learning?



*Answer to question 3:*
- I chose 3 as the number of clusters, based on the value of silhouette score.

- The features I chose are generally representative and can distinguish different groups of univerisities. For example, when the number of component was set to 4, in component 1, the highest weighted feature is graduation rate, the sebset selectivity is also the highest weighted subset. In component 3 and 4, in which the highest weighted subset are both diversity, the highest weighted features (graduate white and undergrad black) are also in this subset.

- Features such as tuition and geography information would be useful. Also, the proportions of student race would tell more stories than the current actual student numbers of races.